# Dictionaries and Sets

## Dict comprehensions

Dictcomps use the same syntax as **listcomps** and **genexps**. A dictcomp builds a `dict` instance by taking `key:value` pairs from any iterable.

In [4]:
dial_codes = [                                         
    (880, 'Bangladesh'),
    (55,  'Brazil'),
    (86,  'China'),
    (91,  'India'),
    (62,  'Indonesia'),
    (81,  'Japan'),
    (234, 'Nigeria'),
    (92,  'Pakistan'),
    (7,   'Russia'),
    (1,   'United States'),
 ]

In [6]:
country_dial = {country: code for code, country in dial_codes}
print(country_dial)

{'Bangladesh': 880, 'Brazil': 55, 'China': 86, 'India': 91, 'Indonesia': 62, 'Japan': 81, 'Nigeria': 234, 'Pakistan': 92, 'Russia': 7, 'United States': 1}


In [7]:
{code: country.upper()
     for country, code in sorted(country_dial.items())
     if code < 70}

{55: 'BRAZIL', 62: 'INDONESIA', 7: 'RUSSIA', 1: 'UNITED STATES'}

## Unpacking Mappings

We can apply `**` to more than 1 argument in a function call. This works only when keys are all strings and unique accross all the arguments

In [15]:
def dump(**kwargs):
    return kwargs

def dump2(x, **kwargs):
    print(x)
    return kwargs

In [13]:
dump(**{'x': 1}, y=2, **{'z': 3, 'v': "next"})

{'x': 1, 'y': 2, 'z': 3, 'v': 'next'}

In [16]:
dump2(**{'x': 1}, y=2, **{'z': 3, 'v': "next"})

1


{'y': 2, 'z': 3, 'v': 'next'}

In [18]:
dump2(**{'z': 1}, y=2, **{'x': 3, 'v': "next"})

3


{'z': 1, 'y': 2, 'v': 'next'}

We can also use `**` inside a `dict` literal, but here duplicate values are allowed - the later occurrences will overide previous ones

In [19]:
{'a': 0, **{'x': 1}, 'y': 2, **{'z': 3, 'x': 4, 'd' : 5}}

{'a': 0, 'x': 4, 'y': 2, 'z': 3, 'd': 5}

## Merging Mappings

Since 3.9 python support `|` adn `|=` to merge mappings, this makes sense because these are also the set union operators. `|` creates a new mapping, while `|=` updates an existing mapping in place.

In [20]:
d1 = {'a': 1, 'b': 3}
d2 = {'a': 2, 'b': 4, 'c': 6}

In [21]:
d1 | d2

{'a': 2, 'b': 4, 'c': 6}

In [22]:
d1

{'a': 1, 'b': 3}

In [23]:
d1 |= d2

In [24]:
d1

{'a': 2, 'b': 4, 'c': 6}

## Pattern Matching with Mappings

The `match/case` statement supports subjects that are mappings, patterns for mappings look like `dict` literals. Destructuring allows to combine and nest different patterns to process records structured like nested mappings and sequences

In [37]:
def get_creators(record: dict) -> list:
    match record:
        case {'type': 'book', 'api': 2, 'authors': [*names]}:
            return names
        case {'type': 'book', 'api': 1, 'author' : name }:
            return [name]
        case {'type': 'book'}:
            raise ValueError(f"invalid book record: {record}")
        case {'type': 'movie', 'director': name}:
            return [name]
        case _: 
            raise ValueError(f'Invalid record: {record}')

In [38]:
b1 = dict(api=1, author='Douglas Hofstadter',
        type='book', title='Gödel, Escher, Bach')
print(get_creators(b1))

['Douglas Hofstadter']


In [39]:
from collections import OrderedDict
b2 = OrderedDict(api=2, type='book',
        title='Python in a Nutshell',
        authors='Martelli Ravenscroft Holden'.split())
print(get_creators(b2))

['Martelli', 'Ravenscroft', 'Holden']


In [41]:
get_creators({'type': 'book', 'pages': 770})

ValueError: invalid book record: {'type': 'book', 'pages': 770}

In mapping patterns there is no need to use `**extra` to match extra key-value pairs, but if we want to capture them we can prefix 1 var with `**` and it must be put as the last in the pattern

In [42]:
food = dict(category='ice cream', flavor='vanilla', cost=199)

match food:
    case {'category': 'ice cream', **details}:
        print(f"details: {details}")

details: {'flavor': 'vanilla', 'cost': 199}


## Standard API of Mapping Types

The `collections.abc` module provides the `Mapping` and `MutableMapping` ABCs which decsribe the interface of dict and similar types.

*The main value of the ABCs is documentign and formalizing the standard interfaces for mappings, and serving as criteria for `isinstance` tests*

*Using `isinstance` with ABC is better than checking if arg is of concrete type e.g. `dict`*

In [49]:
import collections.abc as abc

my_dict = {}
print(isinstance(my_dict, abc.Mapping))
print(isinstance(my_dict, abc.MutableMapping))

True
True


To implement a custom mapping, it's **easier** to extend `collections.UserDict` or to wrap a `dict` by composition instead of subclassing those ABCs

## Inserting and Updating Mutable Values

Accessing `dict` with `d[k]` when `k` is not an existing key raises an `KeyError`. An alternative to `d[k]` is `d.get(k, default)` which returns default value when `k` is not in `dict`

In [1]:
words = ["word", "text", "key", "car", "cat", "word", "key", "taxi", "car", "car", "dog", "cat"]

In [3]:
indecies = {}
for index, word in enumerate(words):
    occurrences = indecies.get(word, [])
    occurrences.append(index)
    indecies[word] = occurrences

for word in sorted(indecies, key=str.upper):
    print(word, indecies[word])

car [3, 8, 9]
cat [4, 11]
dog [10]
key [2, 6]
taxi [7]
text [1]
word [0, 5]


Instead of writing those 3 lines:

```python
occurrences = indecies.get(word, [])
occurrences.append(index)
indecies[word] = occurrences
```

we can replace it with just one line:

```python
indecies.setdefault(word, []).append(index)
```

This is possible because `setdefault` returns the value, so it can be updated without requiring a 2nd search

In other words, this oneliner is equivalent to:

```python
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)
```

but this code would require at least 2 lookups for `key` (3 if it's not found), while `setdefault` require only 1 search of the `key`

In [4]:
indecies = {}
for index, word in enumerate(words):
    indecies.setdefault(word, []).append(index)

for word in sorted(indecies, key=str.upper):
    print(word, indecies[word])

car [3, 8, 9]
cat [4, 11]
dog [10]
key [2, 6]
taxi [7]
text [1]
word [0, 5]


## Automatic Handling of Missing Keys

### defaultdict

A `collections.defaultdict` creates items with default value on demand when a **missing key** is searched using `d[k]` syntax.

When instantiating a `defaultdict` we provide a **callable** to produce a default value whenever `__getitem__` is passed a nonexistent key.

e.g. given `defaultdict` created as `dd = defaultdict(list)`, if `new_key` is not in `dd`, the expression `dd['new_key']` does the following:

1. Calls `list()` to create new list
2. Inserts the list into `dd` using `new_key` as key
3. Returns a reference to that list

**Warning** *This works only for `__getitem__` calls, and not for the other methods. For eg. `dd.get(k)` will still return `None`, and `k in dd` is `False` when k is not in dd*

In [5]:
import collections

indecies = collections.defaultdict(list)
for index, word in enumerate(words):
    indecies[word].append(index)

for word in sorted(indecies, key=str.upper):
    print(word, indecies[word])

car [3, 8, 9]
cat [4, 11]
dog [10]
key [2, 6]
taxi [7]
text [1]
word [0, 5]


### The `__missing__` Method

Underlying the way mappings deal with missing keys is the `__missing__` method. This method is not defined in base `dict` class, but when `dict` is subclassed and provided with `__missing__`, the standard `dict.__getitem__` will call it instead of raising `KeyError` when key is not present.

In [43]:
class StrKeyDict(dict):
    def __missing__(self, key) -> any:
        if isinstance(key, str): # prevents infinite recursion
            raise KeyError(key)
        return self[str(key)]

    def get(self, key, default=None) -> any:
        try:
            return self[key]
        except KeyError:
            return default

    def __contains__(self, key) -> any:
        return (key in self.keys() or str(key) in self.keys())

In [44]:
d = StrKeyDict([('2', 'two'), ('3', 'three')])
print(d['2'])
print(d[3])

print(d.get('2'))
print(d.get('3'))
print(d.get(6, 'N/A'))

print(2 in d)
print('3' in d)
print(1 in d)

two
three
two
three
N/A
True
True
False


## Variations of dict

### collections.OrderedDict

This is nowadays only used to write a backward compatible code to Python versions earlier than 3.6. There are also a few other subtle differences:

* `==` operator checks for matching order
* has a `move_to_end()` method implemented
* `OrderedDict` can handle frequent reordering operations better than `dict`

### collections.ChainMap

A `ChainMap` instance holds a list of mapping which can be searched as 1 mapping, the lookup is performed on each input mapping in order they appear in constructor call. It succeeds as soon as the `key` is found in 1 of those mappings

The `ChainMap` **does not copy** the input mappings, but holds references to them. Updates and insertions to `ChainMap` **only** affect the 1 input mapping.

In [45]:
import collections
d1 = dict(a=1, b=3, d=8)
d2 = dict(a=2, b=4, c=6)

chain = collections.ChainMap(d1, d2)
print(chain['a'])
print(chain['c'])

1
6


In [46]:
chain['c'] = -1
print(d1)
print(d2)

{'a': 1, 'b': 3, 'd': 8, 'c': -1}
{'a': 2, 'b': 4, 'c': 6}


### collections.Counter

A mapping that holds an integer count for each key, updating an existing key adds to its count. 

`Counter` implements `+` and `-` operators to combine tallies, and other usefull methods such as `most_common([n])` 

In [47]:
import collections

counter = collections.Counter("krolkarolkupilkrolowejkaroliniekoralekolorukoralowego")
print(counter)

Counter({'o': 11, 'k': 8, 'l': 8, 'r': 7, 'a': 4, 'e': 4, 'i': 3, 'u': 2, 'w': 2, 'p': 1, 'j': 1, 'n': 1, 'g': 1})


In [48]:
counter.update("alamakota")
print(counter)

Counter({'o': 12, 'k': 9, 'l': 9, 'a': 8, 'r': 7, 'e': 4, 'i': 3, 'u': 2, 'w': 2, 'p': 1, 'j': 1, 'n': 1, 'g': 1, 'm': 1, 't': 1})


In [49]:
counter.most_common(3)

[('o', 12), ('k', 9), ('l', 9)]

In [50]:
counter['q'] += 1 

In [51]:
counter

Counter({'o': 12,
         'k': 9,
         'l': 9,
         'a': 8,
         'r': 7,
         'e': 4,
         'i': 3,
         'u': 2,
         'w': 2,
         'p': 1,
         'j': 1,
         'n': 1,
         'g': 1,
         'm': 1,
         't': 1,
         'q': 1})

### shelve.Shelf

The `shelve` module in stdlib provides persistent storage for a mapping of `string` keys to python objects serialized in the `pickle` binary format.

The `shelve.open` function returns a shelve.Shelf instance - a simple key-value DBM database

* `shelve.Shelf` subclasses `abc.MutableMapping`
* `Shelf` instance is a context manager, so it can be used with `with` block to ensure it is closed after use
* Keys and values are saved whenever new val is assigned to a key
* Keys must be strings
* Values must be objects that `pickle` can serialize


### Subclassing UserDict instead of dict

It is better to create new mapping type be extending `collections.UserDict` rather than `dict`. The main reason for this is that `dict` has some implementations shortcuts that force us to override some methods that we can inherit from `UserDict` without any problems.

`UserDict` does not inherit from `dict`, but uses composition - it has an internal `dict` instance, called `data`, which holds the actual items.

In [52]:
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    def __contains__(self, key):
        return str(key) in self.data
    def __setitem__(self, key, item):
        self.data[str(key)] = item

### Immutable Mappings

The mapping types provided by stdlib are all mutable. The `types` module provides a wrapper class called `MappingProxyType`, which, given a mapping, returns a `mappingproxy` instance that is a read-only, dynamic proxy for the orginal mapping. This means that updates to the orginal mapping can be seen in `mappingproxy`, but changes cannot be made through it.

In [53]:
from types import MappingProxyType

d = {1: 'a'}
d_proxy = MappingProxyType(d)
print(d_proxy)

{1: 'a'}


In [54]:
d_proxy[1]

'a'

In [55]:
d_proxy[2] = 'b'

TypeError: 'mappingproxy' object does not support item assignment

In [57]:
d[3] = 'c'
print(d)
print(d_proxy)

{1: 'a', 3: 'c'}
{1: 'a', 3: 'c'}


## Dictionary Views

The `dict` methods `.keys()`, `.values()` and `.items()` return instances of classes called `dict_keys`, `dict_values` and `dict_items`. These dictionary views are read-only projections of the internal DSs used in `dict` impl. A view is a dynamic proxy.

In [58]:
d = dict(a=10, b=20, c=30)
values = d.values()
print(values)

dict_values([10, 20, 30])


In [59]:
len(values)

3

In [60]:
list(values)

[10, 20, 30]

In [61]:
reversed(values)

In [62]:
values[0]

TypeError: 'dict_values' object is not subscriptable

**Tip** *To save memory, avoid creating instance attributes outside of the `__init__` method when working on objects. This is because python stores all instanc attributes in a common hash table shared by the `__dict__` stored in each new instance that has the same attributes names as the first instance. Adding an instance attribute after `__init__` forces Python to create a new hash table just for the `__dict__` of that 1 instance*

## Set Theory

A set is a collection of unique objects.

In [63]:
l = ['spam', 'spam', 'eggs', 'spam', 'bacon', 'eggs'] 
print(set(l))

{'eggs', 'spam', 'bacon'}


To remove duplicates while preserving the order of the first occurences of each item, a plain `dict` can be used

In [64]:
list(dict.fromkeys(l).keys())

['spam', 'eggs', 'bacon']

Set elements must be hashable. `set` is not hashable but its immutable sibling `frozenset` is hashable.

Set types implement many set operations:

* `a | b` returns their union
* `a & b` computes intersection
* `a - b` returns difference
* `a ^ b` symmetric difference

In [65]:
s1 = {'cat', 'dog', 'penguin', 'zebra', 'lion', 'snake'}
s2 = {'polar bear', 'cat', 'snake', 'gopher'}

In [66]:
s1 | s2

{'cat', 'dog', 'gopher', 'lion', 'penguin', 'polar bear', 'snake', 'zebra'}

In [67]:
s1 & s2

{'cat', 'snake'}

In [68]:
s1 ^ s2

{'dog', 'gopher', 'lion', 'penguin', 'polar bear', 'zebra'}

**Important** *To create an empty `set`, the constructor without args must be used - `set()`. `{}` notation will create an empty `dict`

## Set Comprehensions

In [71]:
from unicodedata import name

{chr(i) for i in range (32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}